In [1]:
# Dependencies
import json
from pprint import pprint
from datetime import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from config import (consumer_key, consumer_secret,
                    access_token, access_token_secret)

In [2]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit=False, wait_on_rate_limit_notify=False)

# Initialize Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Movie csv path
csv_path = 'box_office_data/moviesFinal.csv'

# Read csv
movie_df = pd.read_csv(csv_path)
movie_df

,title,genre,release,rated,value,budget,opening,domestic,worldwide,ratio,zrating,zbudget,zopening,ztotal
0,#Jìyìdàshi,Thriller/Suspense,20170428,R,0.00,20000000,224942,587470,43326383,0.011247,-2.898392,-0.523861,-0.581893,-3.461330
1,#TylerPerrysBoo2AMadeaHalloween,Comedy,20171020,PG-13,0.00,20000000,21226953,47319572,47879572,1.061348,-2.898392,-0.523861,0.033620,102.779753
2,#Gongfuyujia,Action,20170127,PG,0.00,65000000,111979,362657,254569616,0.001723,-2.898392,0.237328,-0.585204,-3.659197
3,#TheEmojiMovie,Adventure,20170728,PG,17.00,50000000,24531923,86089513,216807482,0.490638,-2.055477,-0.016401,0.130480,47.252928
4,#JustGettingStarted,Comedy,20171208,PG-13,22.00,22000000,3201459,6069605,6709821,0.145521,-1.807561,-0.490030,-0.494660,11.265176
5,#Rings,Horror,20170203,PG-13,25.67,25000000,13002632,27793018,82991460,0.520105,-1.625591,-0.439284,-0.207413,49.530827
6,#TheSnowman,Thriller/Suspense,20171020,R,27.00,35000000,3372565,6670765,43232849,0.096359,-1.559645,-0.270131,-0.489645,6.826834
7,#Geostorm,Action,20171020,PG-13,29.33,100000000,13707376,33700160,220800160,0.137074,-1.444116,0.829364,-0.186759,12.719106
8,#FiftyShadesFreed,Drama,20180209,R,29.33,55000000,38560195,98382615,359382615,0.701094,-1.444116,0.068175,0.541612,69.816729
9,#FiftyShadesDarker,Drama,20170210,R,29.67,55000000,46607250,114434010,381081967,0.847405,-1.427258,0.068175,0.777450,84.936272


In [4]:
# # "Real Person" Filters
# min_tweets = 5
# max_tweets = 10000
# max_followers = 2500
# max_following = 2500
# lang = "en"

# Create list of dictionaries
sentiment = []

for title in movie_df['title']:
    
    # Assign title as search term
    search_term = title
    
    # Create variable for holding the oldest tweet
    oldest_tweet = None

    # List to hold average compound values for each movie
    compound_list = []
    
    try:
    
        # Get all tweets from home feed (for each page specified)
        public_tweets = api.search(search_term, 
                                   count=100, 
                                   lang='en', 
                                   result_type="recent", 
                                   max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets['statuses']:
               
            # Use filters to check if user meets conditions
            # if (tweet["user"]["followers_count"] < max_followers and
                # tweet["user"]["statuses_count"] > min_tweets and
                # tweet["user"]["statuses_count"] < max_tweets and
                # tweet["user"]["friends_count"] < max_following and
                # tweet["user"]["lang"] == lang):
            
            # Grab tweet data
            tweet_text = tweet['text']
            
            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            
            # Append compound value to list
            compound_list.append(compound)
            
            # Reassign the the oldest tweet (i.e. the max_id)
            oldest_tweet = int(tweet["id_str"])
        
            # Subtract 1 so the previous oldest isn't included
            # in the new search
            oldest_tweet -= 1

        # Store average 
        tweet_dict = {"title": search_term, 
                      "compound": np.mean(compound_list), 
                      "tweet_count": len(compound_list)}
    
        # Append tweet data to sentiment list
        sentiment.append(tweet_dict)

    except Exception as e:
        print(e)

C:\Users\Justin\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Justin\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
# Create dataframe
movie_sent_df = pd.DataFrame(sentiment)

# Reorder columns
movie_sent_df = movie_sent_df[["title", "compound", "tweet_count"]]

# Today's date
today = dt.strftime(dt.now(), "%Y-%m-%d")

# Save to csv
# movie_sent_df.to_csv(f'all_movie_data/{today}_all_movie_sent.csv', encoding='utf-8', index=False)
movie_sent_df

,title,compound,tweet_count
0,#Jìyìdàshi,NaN,0
1,#TylerPerrysBoo2AMadeaHalloween,NaN,0
2,#Gongfuyujia,NaN,0
3,#TheEmojiMovie,0.019279,47
4,#JustGettingStarted,0.372962,100
5,#Rings,0.221482,62
6,#TheSnowman,0.185384,100
7,#Geostorm,-0.039951,100
8,#FiftyShadesFreed,0.371895,100
9,#FiftyShadesDarker,0.053241,100


In [6]:
# Merge dataframes
merged_df = pd.merge(movie_df, movie_sent_df, how="left")

# Save to csv
# merged_df.to_csv(f'all_movie_data/{today}_all_movie_data.csv', encoding='utf-8', index=False)
merged_df

,title,genre,release,rated,value,budget,opening,domestic,worldwide,ratio,zrating,zbudget,zopening,ztotal,compound,tweet_count
0,#Jìyìdàshi,Thriller/Suspense,20170428,R,0.00,20000000,224942,587470,43326383,0.011247,-2.898392,-0.523861,-0.581893,-3.461330,NaN,0
1,#TylerPerrysBoo2AMadeaHalloween,Comedy,20171020,PG-13,0.00,20000000,21226953,47319572,47879572,1.061348,-2.898392,-0.523861,0.033620,102.779753,NaN,0
2,#Gongfuyujia,Action,20170127,PG,0.00,65000000,111979,362657,254569616,0.001723,-2.898392,0.237328,-0.585204,-3.659197,NaN,0
3,#TheEmojiMovie,Adventure,20170728,PG,17.00,50000000,24531923,86089513,216807482,0.490638,-2.055477,-0.016401,0.130480,47.252928,0.019279,47
4,#JustGettingStarted,Comedy,20171208,PG-13,22.00,22000000,3201459,6069605,6709821,0.145521,-1.807561,-0.490030,-0.494660,11.265176,0.372962,100
5,#Rings,Horror,20170203,PG-13,25.67,25000000,13002632,27793018,82991460,0.520105,-1.625591,-0.439284,-0.207413,49.530827,0.221482,62
6,#TheSnowman,Thriller/Suspense,20171020,R,27.00,35000000,3372565,6670765,43232849,0.096359,-1.559645,-0.270131,-0.489645,6.826834,0.185384,100
7,#Geostorm,Action,20171020,PG-13,29.33,100000000,13707376,33700160,220800160,0.137074,-1.444116,0.829364,-0.186759,12.719106,-0.039951,100
8,#FiftyShadesFreed,Drama,20180209,R,29.33,55000000,38560195,98382615,359382615,0.701094,-1.444116,0.068175,0.541612,69.816729,0.371895,100
9,#FiftyShadesDarker,Drama,20170210,R,29.67,55000000,46607250,114434010,381081967,0.847405,-1.427258,0.068175,0.777450,84.936272,0.053241,100
